# Decorators - exercises

## 1. Function factories

### 1.1 Linear polynomial

Write a function factory that creates linear polynomials. A linear or first degree polynomial is a function that has two unbound parameters ($a$ and $b$) and bound variable ($x$):

\begin{equation*}
f(x) = ax + b.
\end{equation*}

Your function factory should return $f(x)$ for fixed $a$ and $b$ values.

In [1]:
def create_linear_polynomial(a, b):
    
    def fx(x):
        return a * x + b
    
    return fx

f1 = create_linear_polynomial(1, 0)
f2 = create_linear_polynomial(2, -1)

assert f1(3) == 3 # 1*3+0 = 3
assert f2(5) == 9 # 2*5-1 = 9

### 1.2 Higher order polynomial

Create a polynomial factory for arbitrary degree polynomials:

\begin{equation*}
f(x) = a_n  x^n + a_{n-1} x^{n-1} + \dots + a_1 x + a_0.
\end{equation*}

An n-th order polynomial has $n+1$ coefficients including $a_0$.

In [2]:
def create_polynomial(*a):
    
    def fx(x):
        s = a[0]
        for a_i in a[1:]:
            s = x * s  + a_i
        return s
    
    return fx

f1 = create_polynomial(1, 1, 1)
assert f1(2) == 7  # 1*x*x + 1*x + 1 = 7, if x = 2

f2 = create_polynomial(3)
assert f2(10) == 3 and f2(-100) == 3  # constant function

f3 = create_polynomial(3, 2, 1, -10)
assert f3(2) == 24  # 3*x*x*x + 2*x*x + 1*x - 10 = 24 + 8 + 2 - 10 = 24, if x=2

## 2. Decorators without parameters

### 2. 1 Timer decorator

Write a timer decorator that prints how many seconds the wrapped function takes to finish. Make sure that the original return value is returned by the wrapped function and function metadata is kept.

Create both the function (lowercase `timer`) and the class (uppercase `Timer`) version of the decorator.

#### function version

In [3]:
import functools
from datetime import datetime

def timer(func):
    
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        start = datetime.now()
        ret = func(*args, **kwargs)
        duration = (datetime.now() - start).total_seconds()
        print("{} ran for {} seconds".format(func.__name__, duration))
        return ret
    
    return wrapped
    

@timer
def fast_func(x, y):
    """Fast function"""
    for _ in range(1000):
        pass
    print(x + y)
    
@timer
def slow_func():
    for _ in range(10000000):
        pass
    
@timer
def func_with_return(x):
    return 2 * x

fast_func(2, 3)
slow_func()

# metadata should be kept intact
assert fast_func.__doc__ == "Fast function"

# check return value
assert func_with_return(3) == 6

5
fast_func ran for 0.00063 seconds
slow_func ran for 0.275148 seconds
func_with_return ran for 3e-06 seconds


#### class version

In [4]:
class Timer:
    def __init__(self, func):
        self.func = func
        functools.wraps(func)(self)
        
    def __call__(self, *args, **kwargs):
        start = datetime.now()
        ret = self.func(*args, **kwargs)
        duration = (datetime.now() - start).total_seconds()
        print("{} ran for {} seconds".format(self.func.__name__, duration))
        return ret
    

@Timer
def fast_func(x, y):
    """Fast function"""
    for _ in range(1000):
        pass
    print(x + y)
    
@Timer
def slow_func():
    for _ in range(10000000):
        pass
    
@Timer
def func_with_return(x):
    return 2 * x

fast_func(2, 3)
slow_func()

# metadata should be kept intact
assert fast_func.__doc__ == "Fast function"

# check return value
assert func_with_return(3) == 6

5
fast_func ran for 0.00012 seconds
slow_func ran for 0.304604 seconds
func_with_return ran for 4e-06 seconds


### 2.2 `<html>` wrapper

Create a decorator that wraps a function's output between `<html>` and `</html>`. Write a function and a class version as well. You can assume that the wrapped function returns a string.


#### function version

In [5]:
def html_wrapper(func):
    
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        ret = func(*args, **kwargs)
        return "<html>{}</html>".format(ret)
    
    return wrapped

@html_wrapper
def greeter(name):
    return "Hello {}".format(name)

assert greeter("John") == "<html>Hello John</html>"
assert greeter("Peter") == "<html>Hello Peter</html>"

#### class version

In [6]:
class HtmlWrapper:
    def __init__(self, func):
        self.func = func
        functools.wraps(func)(self)
        
    def __call__(self, *args, **kwargs):
        ret = self.func(*args, **kwargs)
        return "<html>{}</html>".format(ret)

@HtmlWrapper
def greeter(name):
    return "Hello {}".format(name)

assert greeter("John") == "<html>Hello John</html>"
assert greeter("Peter") == "<html>Hello Peter</html>"

## 3. Decorators with parameters

You only need to write a function version for the following decorators.

### 3.1 Create and arbitrary tag adder.

The decorator should take the `tag` as its only parameter and wrap the fuction's output between `<tag>` and `</tag>`.

In [7]:
def tag_wrapper(tag):
    def actual_decorator(func):
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            ret = func(*args, **kwargs)
            return "<{0}>{1}</{0}>".format(tag, ret)
        return wrapped
    return actual_decorator

@tag_wrapper("h1")
def greeter(name):
    return "Hello {}".format(name)

@tag_wrapper("span")
def current_time():
    return "{}".format(datetime.now())

assert greeter("John") == "<h1>Hello John</h1>"

ct = current_time()
print(ct)

assert ct.startswith("<span>") and ct.endswith("</span>")

<span>2018-07-25 15:19:01.559688</span>


### 3.2 Parameter type checker

Create a decorator that takes a type as its parameter and checks every argument of the wrapped function including keyword arguments. If any of them are not instances of that type, it should raise a `TypeError`.

In [8]:
def check_type(typ):
    def actual_decorator(func):
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            for i, arg in enumerate(args):
                if not isinstance(arg, typ):
                    raise TypeError("All arguments must be type {}. Positional argument {} is of type {}".format(
                    typ.__name__, i, type(arg)))
            for kw, arg in kwargs.items():
                if not isinstance(arg, typ):
                    raise TypeError("All arguments must be type {}. Keyword argument {} is of type {}".format(
                    typ.__name__, k, type(arg)))
            return func(*args, **kwargs)
        return wrapped
    return actual_decorator
                    

@check_type(int)
def age_printer(age):
    print("I am {} old".format(age))
    
@check_type(int)
def double_age(age):
    return age * 2

@check_type(int)
def compare_age(age1, age2=0):
    return age1 > age2
    
age_printer(1)

assert double_age(12) == 24

try:
    double_age("12")
except TypeError:  # a TypeError should be raised since the function's parameter is not an integer
    pass

assert compare_age(age2=12, age1=11) == False

try:
    compare_age(20, "abc")
except TypeError:
    pass

I am 1 old


### 3.3 Only one exception type

Write a decorator that makes sure that a function only raises one type of error. If it would raise another exception, raise this type instead.

In [9]:
def only_one_exception(exc_type):
    def actual_decorator(func):
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            try:
                ret = func(*args, **kwargs)
            except exc_type:
                raise
            except Exception as e:
                raise exc_type()
            return ret
        return wrapped
    return actual_decorator

def increment_number(x):
    return x + 1  # would raise TypeError if not a number

try:
    increment_number("12")
except TypeError:
    print("TypeError raised")
    
@only_one_exception(ValueError)
def increment_number(x):
    return x + 1  # would raise TypeError if not a number

try:
    increment_number("12")
except ValueError:
    print("ValueError raised")

TypeError raised
ValueError raised


### \*3.4 Argument validator

Create a decorator that takes an arbitrary callable as its parameter and wraps function with one positional parameter. The decorator should 'validate' the parameter with its callable. You can assume that the callable raises a `ValueError` if the parameter is 'invalid', you don't need to return boolean values or raise additional exceptions.

In [10]:
def check_param(validator):
    def actual_decorator(func):
        @functools.wraps(func)
        def wrapped(arg):
            validator(arg)
            return func(arg)
        return wrapped
    return actual_decorator

def positive_int_convertible(n):
    n = int(n)  # raises ValueError if not possible
    if n <= 0:
        raise ValueError("Not a positive integer")
        

@check_param(positive_int_convertible)
def age_printer(age):
    print("Your age is {}".format(int(age)))
        
age_printer(12)
age_printer(12.0)
age_printer("12")

try:
    age_printer("abc")
except ValueError:
    print("ValueError raised, this should happen")

Your age is 12
Your age is 12
Your age is 12
ValueError raised, this should happen


### \*3.5 Check all arguments

In [11]:
import string

def check_all_arguments(*args_checker, **kwargs_checker):
    def actual_decorator(func):
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            for i, arg in enumerate(args):
                args_checker[i](arg)
            for kw, arg in kwargs.items():
                kwargs_checker[kw](arg)
            return func(*args, **kwargs)
        return wrapped
    return actual_decorator

def positive_int_convertible(n):
    n = int(n)  # raises ValueError if not possible
    if n <= 0:
        raise ValueError("Not a positive integer")
        
def hungarian_name_checker(name):
    if not name.istitle():
        raise ValueError("Not a name: {}".format(name))
    non_ascii = set(list("áéíóöőúüű")) 
    letters = non_ascii | set(string.ascii_lowercase) | set(' ')
    name_letters = set(list(name.lower()))
    if name_letters - letters:
        raise ValueError("Name contains non-Hungarian letters: {}".format(name))
        

@check_all_arguments(hungarian_name_checker, positive_int_convertible, height=positive_int_convertible)
def serialize_hun_person_data(name, age, height=160):
    return "Name: {}, age: {}, height: {}".format(name, age, height)
    
assert serialize_hun_person_data("Peter Kovacs", "25", height=180) == "Name: Peter Kovacs, age: 25, height: 180"

icelandic_name = "Aðalheiður"
try:
    serialize_hun_person_data(icelandic_name, "25", height=180)
except ValueError:
    pass

kazakh_name = "Нұрасыл"
try:
    serialize_hun_person_data(kazakh_name, 25)
except ValueError:
    pass

Some exercises are borrowed from various sources:

- https://www.python-course.eu/python3_decorators.php
- https://github.com/manahl/PythonTrainingExercises